# Statoil's Iceberg Classifier

## Problem Statement

Drifting icebergs present threats to navigation and activities in areas such as offshore of the East Coast of Canada.

Currently, many institutions and companies use aerial reconnaissance and shore-based support to monitor environmental conditions and assess risks from icebergs. However, in remote areas with particularly harsh weather, these methods are not feasible, and the only viable monitoring option is via satellite.

Statoil, an international energy company operating worldwide, has worked closely with companies like C-CORE. C-CORE have been using satellite data for over 30 years and have built a computer vision based surveillance system. To keep operations safe and efficient, Statoil is interested in getting a fresh new perspective on how to use machine learning to more accurately detect and discriminate against threatening icebergs as early as possible.

In this competition, you’re challenged to build an algorithm that automatically identifies if a remotely sensed target is a ship or iceberg. Improvements made will help drive the costs down for maintaining safe working conditions.

**We will build a deep neural network that can recognize images with an greater accuracy and less loss function**

**Neural network:**
                    A computational model that works in a similar way to the neurons in the human brain. 
                    Each neuron takes an input,performs some operations then passes the output to the following neuron.
                    [Figure](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)                  

**We will be building a convolutional neural network that will be trained on thousand of trained images data, and later be able to measure on test data to evalute the Test loss and Test accuracy of the builded model.**

The process of building a Convolutional Neural Network always involves four major steps :-

Step - 1 : Convolution

Step - 2 : Pooling

Step - 3 : Flattening

Step - 4 : Full connection

>I used **Keras** which is a simple frameworks where we can initialize the model and keep stacking the layers we want. It makes building deep neural networks very easy

#First let me import **keras** using **Tensorflow** backend.

In [1]:
import keras

Using TensorFlow backend.


In [17]:
#Importing Numpy and Pandas for data analysis...
import numpy as np
import pandas as pd

In [18]:
#Importing SkLearn to split data into Train and Test dataset...
from sklearn.model_selection import train_test_split

Now let me import all the required **keras** packages using which i am going to build our **CNN** model.

In [19]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation, Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten

In [20]:
#Loading the data from json file...
training_data=pd.read_json("Train.json")

In [21]:
#Showcase of data...
training_data.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [22]:
# Input image dimensions...
img_width = 75
img_height = 75

In [23]:
#Defining Epochs, Batch Size & Number of classes...
batch_size = 10
num_classes = 1
epochs = 12

In [24]:
#Generating the training data bands...
Band_1 = np.array([np.array(band).astype(np.float32).reshape(img_height, img_width) for band in training_data["band_1"]])
Band_2 = np.array([np.array(band).astype(np.float32).reshape(img_height, img_width) for band in training_data["band_2"]])

In [25]:
#Create 3 bands having Band_1, Band_2 and avg of both...
X_train = np.concatenate([Band_1[:, :, :, np.newaxis], Band_2[:, :, :, np.newaxis],((Band_1+Band_2)/2)
                          [:, :, :, np.newaxis]], axis=-1)

In [26]:
#Shape of train data...
X_train.shape

(1604, 75, 75, 3)

In [27]:
# The data, shuffled and split between train and test sets...
target_train=training_data['is_iceberg']
x_train, x_test, y_train, y_test = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

C:\Users\HP\Anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [28]:
#Defining shape of input data...
input_shape = ( img_width, img_height, 3)

In [29]:
## Creating our model..
#Initialising the CNN...
model = Sequential()

## Step 1 & Step 2 Convolutional and Pooling..
#Convolutional Layer 1...
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2))

#Convolutional Layer 2...
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2))

#Convolutional Layer 3...
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2))

# Step 3 Flattening..
#Flatten the data for upcoming dense layers...
model.add(Flatten())

# Step 4 Full connection..
#Dense Layer 1...
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

#Sigmoid Layer...
model.add(Dense(num_classes, activation='sigmoid'))

In [30]:
#Compiling the CNN model...
model.compile(loss='binary_crossentropy', optimizer = 'rmsprop', metrics=['accuracy'])

In [33]:
#Fitting the CNN model with train and test data without denoising...
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test,y_test)
            )

Train on 1203 samples, validate on 401 samples
Epoch 1/12
1203/1203 [==============================] - 23s 19ms/step - loss: 0.4303 - acc: 0.7988 - val_loss: 0.4612 - val_acc: 0.8080
Epoch 2/12
1203/1203 [==============================] - 21s 18ms/step - loss: 0.4431 - acc: 0.7880 - val_loss: 0.4442 - val_acc: 0.8155
Epoch 3/12
1203/1203 [==============================] - 20s 16ms/step - loss: 0.4834 - acc: 0.7938 - val_loss: 0.4665 - val_acc: 0.8254
Epoch 4/12
1203/1203 [==============================] - 20s 17ms/step - loss: 0.4443 - acc: 0.8121 - val_loss: 0.6181 - val_acc: 0.7307
Epoch 5/12
1203/1203 [==============================] - 22s 18ms/step - loss: 0.4341 - acc: 0.7997 - val_loss: 0.4442 - val_acc: 0.8504
Epoch 6/12
1203/1203 [==============================] - 20s 16ms/step - loss: 0.4167 - acc: 0.8071 - val_loss: 0.6870 - val_acc: 0.7456
Epoch 7/12
1203/1203 [==============================] - 20s 17ms/step - loss: 0.4402 - acc: 0.8121 - val_loss: 0.4640 - val_acc: 0.8379
E

In [34]:
#Evaluating Test loss and Test Accuracy upon the CNN model...
score = model.evaluate(x_test, y_test, verbose=1)
print('loss:', score[0])
print('Test accuracy:', score[1])

401/401 [==============================] - 4s 10ms/step
Test loss: 0.406430457298
Test accuracy: 0.860349127628


**Conclusion**   
                We may can use more CNN layers and epochs to get the more accuracy and instead of 'rmsprop' may can use 
                'Adam' optimizer for less data loss or error.

**References**

* The Keras Blog and keras.io
* Kaggle - Keras Model for Begineers.
* becominghuman.ai - Building an image classifier using Deep learning.
* tensorflow.org/tutorials/dee_cnn

**Thanks & Regards**

Chandan Shaw

cshawdjp1992@gmail.com
9002800119